# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.widgets import RunDetails

from azureml.automl.runtime.onnx_convert import OnnxConverter

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model, InferenceConfig
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.automl.core.shared import constants

from sklearn.model_selection import train_test_split

import pandas as pd
import os, shutil, joblib
import pickle
import json,requests

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

https://www.kaggle.com/andrewmvd/heart-failure-clinical-data
About this dataset

Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worldwide.

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.

Citation
Davide Chicco, Giuseppe Jurman: Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone. BMC Medical Informatics and Decision Making 20, 16 (2020). 
(link) - https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5

Task - Create a model for predicting mortality caused by Heart Failure.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

https://github.com/MaheshKhatri1960/Udacity-Capstone-Project/blob/master/starter_file/heart_failure_clinical_records_dataset.csv


https://raw.githubusercontent.com/MaheshKhatri1960/Udacity-Capstone-Project/master/heart_failure_clinical_records_dataset.csv
Dataset imported into Workspace and registered as a dataset named 'train_data_2_ds'


In [2]:
ws = Workspace.from_config()
train_data_2_ds = Dataset.get_by_name(ws,name='train_data_2')

In [3]:
#Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json
#ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'MK-1B08-AutoML'

experiment=Experiment(ws, experiment_name)

In [4]:
#MK-1B08

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "MK-1B08-CC"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)


Found existing cluster, use it.
Succeeded...................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
Choice of AutoML settings & configuration - Explained in the README file

In [5]:
automl_settings = {
    "enable_early_stopping" : True,
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=20,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data_2_ds,
    label_column_name="DEATH_EVENT",
    enable_onnx_compatible_models=True,
    n_cross_validations=5,
    debug_log = "automl_errors.log",
    **automl_settings
)

In [6]:
# TODO: Submit your experiment

remote_run = experiment.submit(automl_config, show_output=True)



Running on remote.
No run_configuration provided, running on MK-1B08-CC with default configuration
Running on remote compute: MK-1B08-CC
Parent Run ID: AutoML_1595a0c0-57aa-4749-8bda-bd8ae7747eec

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: ht

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?


In [ ]:
# MK - 1A30 - To Do
OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [8]:
remote_run.wait_for_completion()

{'runId': 'AutoML_1595a0c0-57aa-4749-8bda-bd8ae7747eec',
 'target': 'MK-1B08-CC',
 'status': 'Completed',
 'startTimeUtc': '2021-02-08T12:51:16.127066Z',
 'endTimeUtc': '2021-02-08T13:21:20.928017Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'MK-1B08-CC',
  'AMLSettingsJsonString': '{"path":null,"name":"MK-1B08-AutoML","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-138139","workspace_name":"quick-starts-ws-138139","region":"southcentralus","compute_target":"MK-1B08-CC","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"fea

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
# https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models
    
best_run, fitted_model = remote_run.get_output()
onnx_model= remote_run.get_output(return_onnx_model=True)

print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
print(best_run_metrics)
print(remote_run.summary())


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: MK-1B08-AutoML,
Id: AutoML_1595a0c0-57aa-4749-8bda-bd8ae7747eec_18,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_leaf_nodes=None,
                                                                                                    max_samples=None,
                         

In [10]:
joblib.dump(fitted_model,"fitted_model.joblib")

['fitted_model.joblib']

In [10]:
#TODO: Save the best model
os.makedirs('outputs', exist_ok=True)
joblib.dump(fitted_model, filename="outputs/best_automl_model.pkl")
print("Model saved successfully")

Model saved successfully


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
from azureml.automl.core.shared import constants
model = Model.register(workspace=ws,model_name = "best-automl-model", model_path = './outputs/best_automl_model.pkl')
#, tags={'Method':'AutoML'}, properties={'accuracy': best_run_metrics('accuracy')})

best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

print(model)

print(model.name, model.id, model.version, sep='\t')


Registering model best-automl-model
Model(workspace=Workspace.create(name='quick-starts-ws-138139', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-138139'), name=best-automl-model, id=best-automl-model:1, version=1, tags={}, properties={})
best-automl-model	best-automl-model:1	1


In [ ]:
# MK-1A30 - Reminder - score.py to be loaded before the next cell

In [12]:
env = Environment.from_conda_specification('myenv', 'myenv.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True,tags = {'name':'Heart Failure Prediction-AML'},description='Heart failure prediction model',enable_app_insights = True)
deployed_web_service = model.deploy(workspace=ws,name="automl-deploy-2",models=[model],inference_config=inference_config,deployment_config=aci_config)

deployed_web_service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [13]:
print(deployed_web_service.state)

Healthy


In [14]:
deployed_web_service_uri = deployed_web_service.scoring_uri
deployed_web_service_key = deployed_web_service.get_keys()[0]
print("Deployed Web Service Scoring URI is " + deployed_web_service_uri)
print("Deployed Web Service Scoring URI Authenticaltion Authetication Key: " + deployed_web_service_key)


Deployed Web Service Scoring URI is http://358af297-9c19-45ca-b6d5-7da0ec0010d3.southcentralus.azurecontainer.io/score
Deployed Web Service Scoring URI Authenticaltion Authetication Key: 3XrD5YQXRD1Kkbb0xijWhUBcsV4heNiU


In [20]:
input_data_point_1 = {
  "data": [
    {
                  'age': 55.0, 
                  'anaemia': 0.0, 
                  'creatinine_phosphokinase': 7861.0,
                  'diabetes': 0.0,
                  'ejection_fraction': 38.0,
                  'high_blood_pressure': 0.0,
                  'platelets': 263358.03,
                  'serum_creatinine': 1.1,
                  'serum_sodium': 136.0,
                  'sex': 1.0,
                  'smoking': 0.0,
                  'time': 6.0
    }
 ]
}

input_data = json.dumps(input_data_point_1)
output = deployed_web_service.run(input_data)

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {deployed_web_service_key}'

resp = requests.post(deployed_web_service_uri, input_data, headers=headers)
print("prediction ", resp.text)


input_data_point_2 = {
  "data": [
    {
                  'age': 49.0, 
                  'anaemia': 1.0, 
                  'creatinine_phosphokinase': 80.0,
                  'diabetes': 0.0,
                  'ejection_fraction': 30.0,
                  'high_blood_pressure': 1.0,
                  'platelets': 427000.0,
                  'serum_creatinine': 1.0,
                  'serum_sodium': 138.0,
                  'sex': 0.0,
                  'smoking': 0.0,
                  'time': 12.0
    }
 ]
}


input_data = json.dumps(input_data_point_2)
output = deployed_web_service.run(input_data)

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {deployed_web_service_key}'

resp = requests.post(deployed_web_service_uri, input_data, headers=headers)
print("prediction ", resp.text)



prediction  "{\"result\": [1]}"
prediction  "{\"result\": [0]}"


In [21]:
import json

data = {"data":
        [
          {
            "age": 36,
            "anaemia": 0,
            "creatinine_phosphokinase": 200,
            "diabetes": 0,
            "ejection_fraction": 30,
            "high_blood_pressure": 0,
            "platelets": 120000,
            "serum_creatinine": 1.1,
            "serum_sodium": 135,
            "sex": 1,
            "smoking": 0,
            "time": 7
          },
          {
            "age": 39,
            "anaemia": 0,
            "creatinine_phosphokinase": 300,
            "diabetes": 0,
            "ejection_fraction": 50,
            "high_blood_pressure": 0,
            "platelets": 200000,
            "serum_creatinine": 0.9,
            "serum_sodium": 250,
            "sex": 0,
            "smoking": 0,
            "time": 1
          }
      ]
    }

input_payload = json.dumps(data)

output = deployed_web_service.run(input_payload)

print(output)

{"result": [1, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [17]:
#print the logs of the web service
print(deployed_web_service.get_logs())


2021-02-08T13:49:09,977352069+00:00 - iot-server/run 
2021-02-08T13:49:09,977747075+00:00 - rsyslog/run 
2021-02-08T13:49:09,979560204+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-08T13:49:09,981899141+00:00 - nginx/run 
rsyslogd

In [22]:
#delete the service
deployed_web_service.delete()

In [23]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

